In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from aif360.datasets import AdultDataset, BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    balanced_accuracy_score,
    confusion_matrix
 )

from IPython.display import Markdown, display

sns.set_theme(style="whitegrid")
%matplotlib inline


## Load Dataset

In [ ]:
# privileged and unprivileged groups
privileged_groups = [{'sex': 1, 'age_binary': 1}] # old white males
unprivileged_groups = [{'sex': 0, 'age_binary': 0}]

### Custom preprocessing pipeline
To align with the fairness experiments, we explicitly control how the Adult dataset encodes protected attributes. The block below binarizes age around the dataset median, maps race and sex to binary indicators, and removes the original age column so that downstream models only see the derived protected features.

In [ ]:
# custom processing for the dataset
def custom_preprocessing(df):
    """Binarize age, encode race/sex, and drop raw columns to expose protected attributes explicitly."""
    median_age = df['age'].median()
    df['age_binary'] = (df['age'] > median_age).astype(float)
    df.drop(columns=['age'], inplace=True)
    df['race'] = (df['race'] == 'White').astype(float)
    df['sex'] = (df['sex'] == 'Male').astype(float)
    return df

dataset = AdultDataset(custom_preprocessing=custom_preprocessing,
                              protected_attribute_names=['age_binary', 'sex'],
                              privileged_classes=[np.array([1.0]), np.array([1.0])] ) # old white males

# Get the dataset and split into train and test
np.random.seed(1)
dataset_orig_train, dataset_orig_vt = dataset.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)

#### Clean up training data

In [ ]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

In [ ]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

### Understanding the Metrics

**Important distinction**: The metrics below measure fairness in the **training data labels** (ground truth), NOT in model predictions. 

- **Mean difference** shows whether the actual income labels (>50K vs ≤50K) are distributed differently across privileged vs unprivileged groups in our training data
- A negative value indicates the unprivileged group has fewer positive outcomes (lower income) in the dataset
- This represents **historical bias** in the data itself

In [ ]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

### Apply Reweighing to Balance Training Data

**What Reweighing does:**
- Assigns different weights to training instances based on their group membership and label
- Increases weight for underrepresented combinations (e.g., unprivileged + positive outcome)
- Decreases weight for overrepresented combinations (e.g., privileged + positive outcome)
- Goal: Balance the **label distribution** across groups in the training data

**Important:** This affects how the model learns, but doesn't guarantee the model's predictions will be fair. We'll evaluate prediction fairness later on the test set.

In [ ]:
print('The 50 first instance weights originally:')
print(dataset.instance_weights[:50])

print('The 50 first instance weights after reweighing:')
dataset_transf_train.instance_weights[:50]

In [ ]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

print("\n💡 Interpretation:")
print("   → The mean difference is now ~0, indicating balanced LABEL distribution")
print("   → This means reweighting successfully balanced the training data")
print("   → However, this does NOT yet tell us if the MODEL will make fair predictions")
print("   → We'll evaluate model fairness on test set predictions later")

## Train classifier on original/weighted data

## Load "The Classifier" from Classification Task

Instead of retraining, we load the base classifier saved in Classification.ipynb.
This ensures consistency across all project tasks.

In [ ]:
import joblib

# Load the base classifier trained in Classification.ipynb
artifact = joblib.load("models/the_classifier.joblib")
lr_classifier_origin = artifact["model"]
scaler_original = artifact["scaler"]

print("✓ THE CLASSIFIER loaded successfully")
print(f"  Model type: {type(lr_classifier_origin).__name__}")
print(f"  Feature count: {len(artifact['feature_names'])}")

In [ ]:
# --- Extract features and labels ---
def extract_xy(dataset):
    """Return numpy arrays of features and flattened labels for AIF360 datasets."""
    return dataset.features, dataset.labels.ravel()

In [ ]:
X_train, y_train = extract_xy(dataset_orig_train)
X_valid, y_valid = extract_xy(dataset_orig_valid)
X_test,  y_test  = extract_xy(dataset_orig_test)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled  = scaler.transform(X_test)

X_train_transf, y_train_transf = extract_xy(dataset_transf_train)
w_train_transf = dataset_transf_train.instance_weights
X_train_transf_scaled = scaler.fit_transform(X_train_transf)

In [ ]:
# Use THE CLASSIFIER (loaded from Classification.ipynb) for predictions
# Note: We already loaded lr_classifier_origin above, so we just predict here
y_test_orig_pred = lr_classifier_origin.predict(X_test_scaled)
print("✓ Predictions generated using THE CLASSIFIER (no retraining)")

# Train the FAIR CLASSIFIER on reweighed data using instance weights
lr_classifier_transf = LogisticRegression(max_iter=1000)
lr_classifier_transf.fit(X_train_transf_scaled, y_train_transf,
                         sample_weight=w_train_transf)
y_test_transf_pred = lr_classifier_transf.predict(X_test_scaled)
print("✓ Fair classifier trained with reweighting")

### Theoretical Framework: Fairness Metrics
To quantitatively assess the fairness of our classifier, we utilize several standard metrics from the fair-ML literature. These metrics help us understand how the model treats privileged (e.g., Male, Old) versus unprivileged (e.g., Female, Young) groups.

1.  **Statistical Parity Difference (SPD)**:
    *   **Definition**: The difference in the rate of favorable outcomes received by the unprivileged group compared to the privileged group.
    *   **Formula**: $P(\hat{Y}=1 | D=\text{unprivileged}) - P(\hat{Y}=1 | D=\text{privileged})$
    *   **Interpretation**: A value of 0 implies perfect fairness. Negative values indicate the unprivileged group is less likely to receive a favorable outcome.

2.  **Disparate Impact (DI)**:
    *   **Definition**: The ratio of the rate of favorable outcomes for the unprivileged group to that of the privileged group.
    *   **Formula**: $\frac{P(\hat{Y}=1 | D=\text{unprivileged})}{P(\hat{Y}=1 | D=\text{privileged})}$
    *   **Interpretation**: A value of 1 implies perfect fairness. Values below 1 indicate bias against the unprivileged group (e.g., < 0.8 is often used as a threshold for legal discrimination in US labor law).

3.  **Equal Opportunity Difference (EOD)**:
    *   **Definition**: The difference in True Positive Rates (TPR) between the unprivileged and privileged groups. It focuses on whether qualified individuals are treated equally.
    *   **Formula**: $TPR_{unprivileged} - TPR_{privileged}$, where $TPR = P(\hat{Y}=1 | Y=1)$
    *   **Interpretation**: A value of 0 implies that people who *should* qualify are accepted at the same rate across groups.

4.  **Average Odds Difference (AOD)**:
    *   **Definition**: The average of the difference in False Positive Rates (FPR) and True Positive Rates (TPR) between groups.
    *   **Formula**: $\frac{1}{2} [(FPR_{unprivileged} - FPR_{privileged}) + (TPR_{unprivileged} - TPR_{privileged})]$
    *   **Interpretation**: A value of 0 implies both groups have equal TPR and FPR. This is a stricter metric than Equal Opportunity.

In [ ]:
def summarize_fairness_metrics(metric):
    """Collect commonly used fairness metrics from an AIF360 ClassificationMetric object."""
    return {
        "Statistical Parity Difference": metric.statistical_parity_difference(),
        "Disparate Impact": metric.disparate_impact(),
        "Average Odds Difference": metric.average_odds_difference(),
        "Equal Opportunity Difference": metric.equal_opportunity_difference(),
        "Balanced Accuracy (TPR/TNR avg)": 0.5 * (metric.true_positive_rate() + metric.true_negative_rate())
    }

def plot_model_performance(model_name, y_true, y_pred):
    """Visualize scalar performance metrics and confusion matrix for a classifier."""
    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1": f1_score(y_true, y_pred, zero_division=0)
    }
    metrics_df = pd.DataFrame({
        "Metric": list(metrics.keys()),
        "Value": list(metrics.values())
    })
    cm = confusion_matrix(y_true, y_pred)

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    sns.barplot(data=metrics_df, x="Metric", y="Value", hue="Metric", dodge=False, palette="viridis", ax=axes[0])
    axes[0].set_ylim(0, 1)
    axes[0].set_title(f"{model_name} Metrics")
    axes[0].set_ylabel("Score")
    axes[0].tick_params(axis='x', rotation=45)
    legend = axes[0].get_legend()
    if legend is not None:
        legend.remove()

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[1], 
                xticklabels=['<50k', '>=50k'], yticklabels=['<50k', '>=50k'])
    axes[1].set_title(f"{model_name} Confusion Matrix")
    axes[1].set_xlabel("Predicted")
    axes[1].set_ylabel("Actual")

    plt.tight_layout()
    return metrics

def plot_fairness_comparison(metrics_a, metrics_b, labels=("No Reweighing", "Reweighing")):
    """Plot side-by-side fairness metrics for two models to compare mitigation impact."""
    fairness_df = pd.DataFrame({
        "Metric": list(metrics_a.keys()) + list(metrics_b.keys()),
        "Value": list(metrics_a.values()) + list(metrics_b.values()),
        "Model": [labels[0]] * len(metrics_a) + [labels[1]] * len(metrics_b)
    })

    fig, ax = plt.subplots(figsize=(10, 4))
    sns.barplot(data=fairness_df, x="Metric", y="Value", hue="Model", ax=ax)
    ax.set_title("Fairness Metric Comparison")
    ax.tick_params(axis='x', rotation=45)
    ax.set_ylabel("Value")
    ax.axhline(0, color='black', linewidth=0.8)
    plt.tight_layout()

In [ ]:
# Confirm label association: numeric <-> original categories
print('\n### Label association checks')

# Numeric label values and favorable/unfavorable mapping from AIF360 dataset
print('Unique numeric labels in training set (np.unique):', np.unique(dataset_orig_train.labels))
print('Favorable label (AIF360):', dataset_orig_train.favorable_label)
print('Unfavorable label (AIF360):', dataset_orig_train.unfavorable_label)
print('Label names (if present in DataFrame):', dataset_orig_train.label_names)

# Convert test set to pandas DataFrame to inspect original (string) label values
df_test, meta = dataset_orig_test.convert_to_dataframe()
label_col = dataset_orig_test.label_names[0] if len(dataset_orig_test.label_names) > 0 else None

if label_col and label_col in df_test.columns:
    print(f"\nLabel column in DataFrame: {label_col}")
    print('Unique string values in label column:', df_test[label_col].unique())
    display(pd.DataFrame(df_test[label_col].value_counts()).rename(columns={label_col: 'count'}))

    # Show sample rows for each label to confirm mapping
    print('\nSample rows for unfavorable label (numeric =', dataset_orig_test.unfavorable_label, ')')
    display(df_test[df_test[label_col] == dataset_orig_test.unfavorable_label].sample(min(3, len(df_test))))
    print('\nSample rows for favorable label (numeric =', dataset_orig_test.favorable_label, ')')
    display(df_test[df_test[label_col] == dataset_orig_test.favorable_label].sample(min(3, len(df_test))))
else:
    # As a fallback, display the head of the DataFrame together with numeric labels
    print('\nNo explicit label column found in converted DataFrame. Showing numeric labels beside the first few rows:')
    head_df = df_test.head().copy()
    # the BinaryLabelDataset stores labels as a 2D numpy array; flatten for display
    head_df['numeric_label'] = dataset_orig_test.labels[:head_df.shape[0], 0]
    display(head_df)

# Show classifier classes and unique predictions to ensure they are using the same numeric mapping
print('\nClassifier classes (original):', getattr(lr_classifier_origin, 'classes_', None))
print('Classifier classes (reweighed):', getattr(lr_classifier_transf, 'classes_', None))
print('Unique predictions (original) observed on test set:', np.unique(y_test_orig_pred))
print('Unique predictions (reweighed) observed on test set:', np.unique(y_test_transf_pred))

# Quick consistency checks (print results rather than raising) to avoid halting the notebook
print('\nConsistency checks:')
print('Does classifier classes include the dataset favorable/unfavorable numeric values? ',
      set(getattr(lr_classifier_origin, 'classes_', [])) >= set(np.unique(dataset_orig_train.labels)))
print('Do predictions only use numeric labels present in the dataset? ', set(np.unique(y_test_orig_pred)) <= set(np.unique(dataset_orig_train.labels)))


In [ ]:
# Evaluate models
print("MODEL EVALUATION DASHBOARD")

# Visual diagnostics for the original and reweighed models
metrics_orig = plot_model_performance("No Reweighing", y_test, y_test_orig_pred)
metrics_transf = plot_model_performance("Reweighing", y_test, y_test_transf_pred)

# Fairness metrics for both models
dataset_orig_test_pred = dataset_orig_test.copy()
dataset_orig_test_pred.labels = y_test_orig_pred
metric_orig_test = ClassificationMetric(dataset_orig_test, dataset_orig_test_pred,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)
fairness_metrics_orig = summarize_fairness_metrics(metric_orig_test)

dataset_transf_test_pred = dataset_orig_test.copy()
dataset_transf_test_pred.labels = y_test_transf_pred
metric_transf_test = ClassificationMetric(dataset_orig_test, dataset_transf_test_pred,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=privileged_groups)
fairness_metrics_transf = summarize_fairness_metrics(metric_transf_test)

# Compare fairness metrics side-by-side with a bar plot and styled table
plot_fairness_comparison(fairness_metrics_orig, fairness_metrics_transf)
fairness_table = pd.DataFrame({
    "Metric": list(fairness_metrics_orig.keys()),
    "No Reweighing": list(fairness_metrics_orig.values()),
    "Reweighing": list(fairness_metrics_transf.values())
})
display(fairness_table.style.format({"No Reweighing": "{:.4f}", "Reweighing": "{:.4f}"}).set_caption("Fairness metrics summary"))

## Comparison: The Classifier vs Fair Classifier

We now compare **THE CLASSIFIER** (base model from Classification task) with the **Fair Classifier** (reweighted model) in terms of both performance and fairness metrics.

In [ ]:
print("=" * 70)
print("COMPARISON: THE CLASSIFIER vs FAIR CLASSIFIER")
print("=" * 70)
print(f"\nPerformance Metrics:")
print(f"  THE CLASSIFIER Accuracy:    {accuracy_score(y_test, y_test_orig_pred):.4f}")
print(f"  Fair Classifier Accuracy:   {accuracy_score(y_test, y_test_transf_pred):.4f}")
print(f"  Accuracy Difference:        {accuracy_score(y_test, y_test_transf_pred) - accuracy_score(y_test, y_test_orig_pred):.4f}")

print(f"\nFairness Metrics (Statistical Parity Difference):")
print(f"  THE CLASSIFIER SPD:         {metric_orig_test.statistical_parity_difference():.4f}")
print(f"  Fair Classifier SPD:        {metric_transf_test.statistical_parity_difference():.4f}")
print(f"  SPD Improvement:            {abs(metric_transf_test.statistical_parity_difference()) - abs(metric_orig_test.statistical_parity_difference()):.4f}")

print("\n✓ Note: Statistical Parity Difference closer to 0 indicates better fairness")
print("=" * 70)

### Results analysis
> 
> The visual dashboard highlights that the reweighted model sacrifices roughly two percentage points of accuracy and balanced accuracy relative to the baseline, yet the fairness bar chart and summary table show markedly narrower disparities. In particular, statistical parity difference improves from −0.36 to −0.14 and disparate impact moves closer to the ideal value of one, while average odds and equal opportunity differences shrink toward zero. These shifts corroborate the SPD comparison table beneath the plots: although the mitigated classifier is slightly less predictive, it delivers substantively fairer outcomes across protected groups, demonstrating the classic utility–fairness trade-off.

### Fairness requirements checklist
- **Protected attributes**: Age (binarized) and Sex are explicitly encoded and tracked through AIF360 datasets, satisfying the requirement to focus on these groups.
- **Classifier + fairness metric**: A baseline logistic regression model is trained and evaluated with statistical parity difference, disparate impact, average odds difference, and equal opportunity difference.
- **Mitigation technique**: Reweighing is applied to the training data, producing a fairer model whose metrics are directly compared to the baseline.
- **Reporting**: Visual dashboards plus the comparison table document the before/after fairness metrics, showing measurable improvement even if SPD is not exactly zero.

✅ *Conclusion:* the fairness analysis portion of the project statement (pre-privacy) is complete. Next steps will involve repeating the same metric/mitigation workflow on the privacy-preserving dataset once it is available.

## Save the Fair Classifier

Save the reweighted fair classifier for potential reuse in subsequent tasks.

In [ ]:
import os
import joblib

os.makedirs("models", exist_ok=True)

# Save the fair classifier artifact
fair_artifact = {
    "model": lr_classifier_transf,
    "scaler": scaler,
    "feature_names": list(dataset_orig_train.feature_names),
    "reweighing": RW  # Include the reweighing transformer
}

joblib.dump(fair_artifact, "models/the_fair_classifier.joblib")
print("✓ Fair classifier saved as: models/the_fair_classifier.joblib")